In [1]:
#use wget to get bert model

import os
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip

if "bert" not in os.listdir():
  os.makedirs("bert")
else:
  pass

import zipfile
import shutil
         
bert_zip = zipfile.ZipFile('multi_cased_L-12_H-768_A-12.zip')
bert_zip.extractall('bert')
 
bert_zip.close()

def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

copytree("bert/multi_cased_L-12_H-768_A-12", "bert")

--2021-05-02 01:58:48--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 34.64.4.112, 34.64.4.16, 34.64.4.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|34.64.4.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

        multi_cased  52%[=========>          ] 332.01M  15.1MB/s    eta 21s    ^C


BadZipFile: File is not a zip file

In [ ]:
#using os to import bert model
import os

os.listdir('')

In [ ]:
#import requirements
import tensorflow as tf

import pandas as pd
import numpy as np 
import re
import pickle

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

In [ ]:
#install bert model
%pip install keras-bert
%pip install keras-radam

In [ ]:
#import model, activationfunction, tokenizer
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam

In [ ]:
train = pd.read_table("경로"+"파일명.txt")
test = pd.read_table("경로"+"파일명.txt")

In [ ]:
#setting model dictionary
'''
SEQ_LEN = 문장 최대 길이 
SEQ_LEN 이 최대 길이보다 작다면 남은 부분은 0으로 채워지고, 길다면 자름
BATCH_SIZE = 16으로 정한 후 조정
EPOCHS = 2 -> 대부분 에폭2단에서 좋은 결과물을 추출함
Learning Rate는 1e-5로 초기설정 후 튜닝
pretrained_bert 는 사전학습 모델이 있는 폴더를 의미
긍정 및 부정인지 알려주는 칼럼을 label로 설정
'''
SEQ_LEN = 800
BATCH_SIZE = 16
EPOCHS = 2
LR = 1e-5

pretrained_path ="bert"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

DATA_COLUMN = "document"
LABEL_COLUMN = "label"

'''
vocab.txt에 있는 단어에 인덱스를 추가해주는 token_dict 딕셔너리 생성
1. 우리가 분석할 문장이 토큰화
2. 인덱스로 변경되어 BERT 신경망에 인풋으로 입력
'''

token_dict = {}
with codes.open(vocab_path, 'r', 'utf8') as reader:
	for line in reader:
		token = line.strip()
		if "_" in token:
			token = token.replace("_", "")
			token = '##' + token
		token_dict[token] = len(token_dict)

'''
1. Tokenizer 상속 후 inherit_Tokennizer 클래스 재정의
2. _tokenize 함수를 작성
why? -> 영어식으로 쪼개지기때문에 한글화 시키기 위함
inherit_Tokennizer는 문장을 토큰화
단어의 시작이 아닌 부분엔 ##으로 대체
'''
class inherit_Tokenizer(Tokenizer):
  def _tokenize(self, text):
        if not self._cased:
            text = text
            
            text = text.lower()
        spaced = ''
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
        tokens = []
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
        return tokens


    tokenizer = inherit_Tokenizer(token_dict)

In [ ]:

'''
데이터를 BERT에 맞는 형식으로 변형해주는 함수 정의
tokenizer.encode 함수가 BERT 모형을 토큰화하고, 토큰화된 단어를 인덱스에 맞게 숫자로 변경
'''
def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        targets.append(data_df[LABEL_COLUMN][i])
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)

    data_x, data_y = convert_data(data_df)

    return data_x, data_y

train_x, train_y = load_data(train)
test_x, test_y = load_data(test)

'''
Pre-trained Model의 Input은 토큰화과정에서 숫자로 변형된 것
그리고 앞문장인지 뒷문장인지 알려주는지 문장 순서 벡터가 들어감.
파인튜닝 시 마스킹을 사용하지 않음
'''

In [ ]:
'''
<BERT input을 문장으로 쓸 경우>
윗 설명 예시 코드
'''
def sentence_convert_data(data):
    global tokenizer
    indices = []
    for i in tqdm(range(len(data))):
        print(tokenizer.tokenize(data[i]))
        ids, segments = tokenizer.encode(data[i], max_len=SEQ_LEN)
        indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def sentence_load_data(sentences):#sentence는 List로 받는다
           
    data_x = sentence_convert_data(sentences)

    return data_x

sentense_load_data(["버트 테스트 123 123 123", "버트 테스트야 개꿀"])

In [ ]:
#pre-trained model load
layer_num = 12
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,)

model.summary() #모델 요약

In [ ]:
#model tuning
'''
사전학습 모델 변형
input : 토큰 벡터, 세그먼트
위와 같은 이유로 inputs = model.inputs[:2]로 정의

사전 모델을 변형
Dense(1)을 사전 학습 모델에 추가 -> 긍정인지 부정인지 알려줌
Dense(1) : 긍정에 가까우면 0, 부정은 1
Radam을 사용해 Gradient Descent training
output : bert_model
'''

def get_bert_finetuning_model(model):
  inputs = model.inputs[:2]
  dense = model.layers[-3].output


  outputs = keras.layers.Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)



  bert_model = keras.models.Model(inputs, outputs)
  bert_model.compile(
      optimizer=RAdam(learning_rate=0.00001, weight_decay=0.0025),
      loss='binary_crossentropy',
      metrics=['accuracy'])
  
  return bert_model 

In [ ]:
#model flow check
'''
<선택>
모델 flow 확인
'''
from IPython.display import SVG
from keras.utils import model_to_dot


SVG(model_to_dot(get_bert_finetuning_model(model), dpi=65).create(prog='dot', format='svg'))

In [ ]:
#load & fine tuning
'''
bert_model을 get_bert_finetuning_model 함수로 load
bert_model.fit -> 파인튜닝 시작
validation_data = (test_x,test_y) -> 실시간으로 테스트 데이터에 대한 정확도를 알기 위함
'''
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init = tf.variables_initializer([v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables])
sess.run(init)

bert_model = get_bert_finetuning_model(model)
history = bert_model.fit(train_x, train_y, epochs=2, batch_size=16, verbose = 1, validation_data=(test_x, test_y), shuffle=True)

#모델 저장
bert_model.save_weights(path+"/bert.h5")

In [ ]:
#bert model load and predict definition
'''
BERT Model load
'''
bert_model = get_bert_finetuning_model(model)
bert_model.load_weights(path+"/bert.h5")

'''
F1 Score확인
Real   Data
긍정 -> 진짜 긍정 확률
부정 -> 부정 확률

predict_convert_data : 테스트 데이터를 input하기 위함. test데이터이기에 label 고려 X
'''

def predict_convert_data(data_df):
    global tokenizer
    indices = []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        
    items = indices
    
    
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def predict_load_data(x): #Pandas Dataframe을 인풋으로 받는다
    data_df = x
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)

    return data_x

test_set = predict_load_data(test)

In [ ]:
#Predict
preds = bert_model.predict(test_set)

# 부정 -> 0, 긍정 -> 1 출력
preds

In [ ]:
y_true = test['label']
# F1 Score 확인
print(classification_report(y_true, np.round(preds,0)))

In [ ]:
'''
문장이 입력되면 긍정인지 부정인지 알려주는 함수
'''
def sentence_convert_data(data):
    global tokenizer
    indices = []
    ids, segments = tokenizer.encode(data, max_len=SEQ_LEN)
    indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def sentence_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = bert_model.predict(data_x)
    predict_answer = np.round(np.ravel(predict), 0).item()
    
    if predict_answer == 0:
      print("부정적")
    elif predict_answer == 1:
      print("긍정적")

# 데이터 준비
+ 기사번호 기사문장 부정적0/긍정적1
+ 같은 형식으로 제작해야할듯
+ 크롤링 후 수작업 하는 식으로 진행해야할듯